In [ ]:
import os

In [ ]:
import tensorflow as tf

In [ ]:
import sys

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
from scipy.ndimage.filters import gaussian_filter1d

In [ ]:
sys.version

In [ ]:
!pip --version

# test

In [ ]:
os.listdir("tensorboard_logs")

In [ ]:
def read_tf_data(model, d):
    f = 'tensorboard_logs/test-{}-ERSF-k1-d={}-ensemble'.format(model, d)
    # assume only one file in each folder
    assert len(os.listdir(f)) == 1
    f = os.path.join(f, os.listdir(f)[0])
    it = tf.compat.v1.train.summary_iterator(f)
    
    # the first seems to be a placeholder
    first = next(it)
    assert first.step == 0
    assert len(first.summary.value) == 0

    steps = []
    precs = []
    recalls = []
    
    for e in it:
        v = e.summary.value[0]
        if v.tag == 'graph/prec':
            steps.append(e.step)
            precs.append(v.simple_value)
        elif v.tag == 'graph/recall':
            recalls.append(v.simple_value)
        else:
            pass
    res = {}
    steps = np.array(steps)
    precs = np.array(precs)
    recalls = np.array(recalls)
    res['steps'] = steps
    res['precs'] = precs
    res['recalls'] = recalls
    return res

In [ ]:
fc10 = read_tf_data('FC', 10)

In [ ]:
fc10['steps']

In [ ]:
fc20 = read_tf_data('FC', 20)
fc50 = read_tf_data('FC', 50)
cnn20 = read_tf_data('CNN', 20)
cnn50 = read_tf_data('CNN', 50)
eq20 = read_tf_data('EQ2', 20)
eq50 = read_tf_data('EQ2', 50)

In [ ]:
# plot it
x = np.linspace(steps.min(), steps.max(), 20)
spl = make_interp_spline(steps, precs, k=3)  # type: BSpline
y = spl(x)

plt.plot(x, y)

In [ ]:
def smoothed_plot(ax, x, y, label=''):
    xnew = np.linspace(x.min(), x.max(), 100)
    spl = make_interp_spline(x, y, k=3)  # type: BSpline
    ynew = spl(xnew)
    ax.plot(xnew, ynew, label=label)

In [ ]:
def smoothed_plot(ax, x, y, label=''):
    ax.plot(x, y, label=label)

In [ ]:
def plot_subfig(ax, d, which, eq, fc, cnn):
    # smoothing
    smoothed_plot(ax, eq['steps'], eq[which], label='EQ')
    smoothed_plot(ax, fc['steps'], fc[which], label='FC')
    smoothed_plot(ax, cnn['steps'], cnn[which], label='CNN')
    ax.set_xlabel('steps')
    ax.set_ylabel(which)
    ax.legend()
    ax.set_title('{} for d={}'.format(which, d))

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,10))
plot_subfig(axs[0,0], 20, 'precs', eq20, fc20, cnn20)
plot_subfig(axs[0,1], 20, 'recalls', eq20, fc20, cnn20)
plot_subfig(axs[1,0], 50, 'precs', eq50, fc50, cnn50)
plot_subfig(axs[1,1], 50, 'recalls', eq50, fc50, cnn50)
plt.savefig('a.pdf')